## To do:

# Notebook for scraping different regions

I created a separate notebook to scrape different regions. This allows me to collect games without storing the JSON.

Load libraries

In [2]:
import requests, json
import numpy as np
from src import API_io
import importlib
import pandas as pd
import time
from src import feature_calc

In [3]:
working_dir = 'C:\\Users\\Me\\Documents\\GitHub\\lolML'
with open(working_dir+ '\\api_key.txt', 'r') as api_file:
    api_key =  api_file.read()

## Get a list of high level matches

Set the region. Then get summoners in featured games for that region

In [4]:
region_key = 'na'
featured_json = API_io.load_featured_games(api_key, region_key) # load json of featured games
featured_game_ids = [x['gameId'] for x in featured_json ] #  use list comprehension to get featured games; don't use this

Make a list of summoner names and summoner IDs from the featured JSON

In [50]:
importlib.reload(API_io)
summoner_names, summoner_IDs = API_io.get_summoners_IDs_from_featured_games(featured_json, api_key, region =region_key)
print(summoner_names[:5],)

['KoreaNidalee', 'InitialLotus', 'Adriana Ugarte', 'PuffyCheeks', 'chuck normis']


In [51]:
summoner_urls = [API_io.make_matchlist_url_summoner_ID(x, True, True, api_key, region =region_key) for x in summoner_IDs]
print(summoner_urls[:2])
match_histories = [API_io.get_limited_request(x) for x in summoner_urls ]

['https://na.api.pvp.net/api/lol/na/v2.2/matchlist/by-summoner/31756933?rankedQueues=RANKED_SOLO_5x5&seasons=SEASON2015&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7', 'https://na.api.pvp.net/api/lol/na/v2.2/matchlist/by-summoner/20880807?rankedQueues=RANKED_SOLO_5x5&seasons=SEASON2015&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7']


Extract the match ID from the match history JSON

In [52]:
match_IDs = np.empty(0, dtype=int)
for cur_matches in match_histories:
    match_IDs = np.append( match_IDs, [x['matchId'] for x in cur_matches['matches'] if x['region'] == region_key.upper()] )
match_IDs = np.unique(match_IDs)
pd.Series(match_IDs).to_csv('Korean Match IDs.csv')
match_IDs.shape

(35303,)

In [21]:
# make urls for loading
match_urls = [API_io.make_match_info_url(x, True, api_key, region = region_key) for x in match_IDs] # True flag means we get the timeline
match_urls[:2]

['https://euw.api.pvp.net/api/lol/euw/v2.2/match/1930396612?includeTimeline=true&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7',
 'https://euw.api.pvp.net/api/lol/euw/v2.2/match/1930399900?includeTimeline=true&api_key=0da3703d-7bf5-4e72-96cd-5062b28720d7']

In [171]:
match_range = np.arange(10000, 15000)
col_names = feature_calc.col_names
timeline_end = 55
time_indices = np.arange(5, timeline_end, 5)
games_df = pd.DataFrame(index = match_range, columns= col_names)
timelines_df = [ pd.DataFrame(index = match_range, columns= col_names) for x in time_indices]

In [172]:
# this is the worst code EVER
for match_index, cur_match in enumerate(match_range):
    try:
        match_info = requests.get(match_urls[cur_match]).json()
        for time_index, last_min in enumerate(time_indices):
            try:
                timelines_df[time_index].loc[match_index] = feature_calc.calc_features_single_match(match_info, last_min)
            except:
                continue
    except requests.exceptions.HTTPError as err:
        print('HHTPError in game ' + str(match_IDs[cur_match]))
    except:
        import sys
        err = sys.exc_info()[0]
        print('Error: ' + str(err) + ' in game ' + str(match_IDs[cur_match]))
    time.sleep(0.3) # RIOT API is throttled to ~0.83 requests / second

Error: <class 'KeyboardInterrupt'> in game 2077600049
Error: <class 'KeyboardInterrupt'> in game 2077609330
Error: <class 'KeyboardInterrupt'> in game 2077618658
Error: <class 'KeyboardInterrupt'> in game 2077623710
Error: <class 'KeyboardInterrupt'> in game 2077642154
Error: <class 'KeyboardInterrupt'> in game 2077643397
Error: <class 'KeyboardInterrupt'> in game 2077648353


KeyboardInterrupt: 

In [138]:
timelines_df = [x.dropna() for x in timelines_df]
timelines_df = [feature_calc.retype_columns(x) for x in timelines_df ]
timelines_df = [x.set_index('matchId') for x in timelines_df ]

In [139]:
[x.shape for x in timelines_df]

[(4694, 31),
 (4688, 31),
 (4648, 31),
 (4546, 31),
 (3837, 31),
 (2995, 31),
 (1880, 31),
 (889, 31),
 (327, 31),
 (82, 31)]

Now save the information to disk.

In [59]:
games_df.to_csv('Korean games 1-1000.csv')

In [169]:
combined_df = [timelines_df[x].append(combined_df[x] ) for x in np.arange(len(timelines_df)) ]

In [170]:
[x.shape for x in combined_df]

[(9182, 31),
 (9176, 31),
 (9108, 31),
 (8926, 31),
 (7567, 31),
 (5929, 31),
 (3624, 31),
 (1719, 31),
 (571, 31),
 (134, 31)]

In [174]:
import pickle
with open('combined_df.pickle', 'wb') as pickle_file:
    pickle.dump(combined_df, pickle_file)

In [177]:
with open('combined_df.pickle', 'rb') as pickle_file:
    combined_df = pickle.load(pickle_file)